# UAM Validation 05 — Deterministic SN + BAO (Delta & alpha analytically marginalized)

- SN: Pantheon+SH0ES full covariance with analytic Delta-marginalization.
- BAO: Uses curated 'bao_spec.json'. Theory predicts (X/rd) = alpha * g(z) with a single global alpha = (c/H0)/rd marginalized analytically across all BAO blocks.
- Models: UAM (no cosmology parameter), LCDM (Omega_m grid). Nuisance parameters (Delta for SN, alpha for BAO) are marginalized and not counted in k.
- Outputs: chi2_SN, chi2_BAO, chi2_tot, AIC, BIC, Omega_m(best), logs, and residual counts.


In [ ]:

import os, sys, json, time, math, re, platform
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import cho_factor, cho_solve
import importlib

ROOT     = Path("/content/notebook")
DATA_DIR = ROOT/"GoldenMaster"/"data_curated"
RES_DIR  = ROOT/"UAM_validations"/"results"/"sn_bao"
LOG_DIR  = ROOT/"logs"
RES_DIR.mkdir(parents=True, exist_ok=True); LOG_DIR.mkdir(parents=True, exist_ok=True)

# ---- Load SN table + covariance (same as baseline) ----
CFG = json.loads((DATA_DIR/"column_map.json").read_text())
TABLE = DATA_DIR/CFG["table"]
COV   = DATA_DIR/CFG["cov"]
ZCOL, MUCOL = CFG["z_col"], CFG["mu_col"]

df = pd.read_csv(TABLE, sep=r"\s+", comment="#", engine="python")
z_all  = df[ZCOL].to_numpy(float)
mu_all = df[MUCOL].to_numpy(float)
mask = (z_all>=0.0) & (z_all<=2.3) & np.isfinite(mu_all)
z_sn, mu_obs = z_all[mask], mu_all[mask]
idx = np.where(mask)[0]
N_sn = len(z_sn)

flat = np.loadtxt(COV).ravel().astype(float)
if flat.size == len(mu_all)*len(mu_all)+1: flat = flat[1:]
C_full = flat.reshape((len(mu_all),len(mu_all)), order="C")
C_full = 0.5*(C_full + C_full.T)
C = C_full[np.ix_(idx, idx)]
C = 0.5*(C + C.T)

# ---- Cholesky factor (SN) ----
def spd_cholesky(C):
    try:
        return cho_factor(C, lower=False, check_finite=False), {"method":"none"}
    except Exception:
        pass
    for jit in [1e-12,1e-11,1e-10,1e-9]:
        try:
            return cho_factor(C + jit*np.eye(C.shape[0]), lower=False, check_finite=False), {"method":"jitter","jitter":jit}
        except Exception: pass
    w,V = np.linalg.eigh(C); floor = max(1e-12, 1e-12*abs(np.median(np.diag(C))), 1e-12*abs(np.median(w)))
    w = np.maximum(w, floor); Cc = (V*w)@V.T; Cc = 0.5*(Cc + Cc.T)
    return cho_factor(Cc, lower=False, check_finite=False), {"method":"eigen_clip","floor":float(floor)}
Cf_sn, spd_sn = spd_cholesky(C)
def Ci_sn(v): return cho_solve(Cf_sn, v, check_finite=False)

# ---- Theory import ----
sys.path.insert(0, str(ROOT))
theory = importlib.import_module("GoldenMaster.theory.uam_model")
E2_uam = theory.E2_uam
def E2_lcdm_factory(om): return lambda zz: om*(1+np.asarray(zz,float))**3 + (1-om)

# ---- Distance integrators ----
def mu_from_E2_grid(E2_func, z_eval, n_grid=20000):
    z_eval = np.asarray(z_eval, float)
    if z_eval.size==0: return z_eval.copy()
    zmax = float(np.max(z_eval))
    grid = np.linspace(0.0, zmax, int(n_grid))
    invE = 1.0/np.sqrt(np.clip(E2_func(grid), 1e-300, np.inf))
    # trapezoid cumulative integral
    csum = np.cumsum((invE[1:]+invE[:-1])*(grid[1:]-grid[:-1])*0.5)
    Dc = np.empty_like(grid); Dc[0]=0.0; Dc[1:]=csum
    DL = (1.0+grid)*Dc
    return 5.0*np.log10(np.clip(np.interp(z_eval, grid, DL), 1e-300, np.inf)) + 25.0

def I_of_z(E2_func, z_eval, n_grid=20000):
    z_eval = np.asarray(z_eval, float)
    if z_eval.size==0: return z_eval.copy()
    zmax = float(np.max(z_eval))
    grid = np.linspace(0.0, zmax, int(n_grid))
    invE = 1.0/np.sqrt(np.clip(E2_func(grid), 1e-300, np.inf))
    csum = np.cumsum((invE[1:]+invE[:-1])*(grid[1:]-grid[:-1])*0.5)
    Dc = np.empty_like(grid); Dc[0]=0.0; Dc[1:]=csum
    return np.interp(z_eval, grid, Dc)

def DM_over_rd_g(E2_func, z):
    return I_of_z(E2_func, z)                      # g_DM(z)
def DH_over_rd_g(E2_func, z):
    z = np.asarray(z,float)
    return 1.0/np.sqrt(np.clip(E2_func(z), 1e-300, np.inf))  # g_DH(z)=1/E
def DV_over_rd_g(E2_func, z):
    I = I_of_z(E2_func, z); Ez = np.sqrt(np.clip(E2_func(z), 1e-300, np.inf))
    return np.power(np.clip(I*I * (np.asarray(z,float)/Ez), 0.0, np.inf), 1.0/3.0)  # g_DV(z)

# ---- SN analytic Delta-marg chi^2 ----
one_sn = np.ones(N_sn)
def chi2_sn_marg(mu_model):
    r = mu_obs - mu_model
    A = float(one_sn @ Ci_sn(one_sn)); B = float(one_sn @ Ci_sn(r))
    return float(r @ Ci_sn(r) - (B*B)/A)

def delta_hat(mu_model):
    r = mu_obs - mu_model
    A = float(one_sn @ Ci_sn(one_sn)); B = float(one_sn @ Ci_sn(r))
    return B/A

# ---- BAO loader per spec ----
SPEC = DATA_DIR/"bao_spec.json"
if not SPEC.exists():
    print(f"[BAO] Spec missing: {SPEC}. Proceeding with SN-only mode.")
    spec = {"entries": []}
spec = json.loads(SPEC.read_text())
entries = spec.get("entries", [])
if not entries:
    print("[BAO] Spec has no entries; proceeding with SN-only mode.")
    blocks = []
def load_matrix(path, N_expect=None):
    arr = np.loadtxt(path).ravel().astype(float)
    if N_expect is not None and arr.size == N_expect*N_expect + 1:
        arr = arr[1:]
    if N_expect is not None and arr.size != N_expect*N_expect:
        raise ValueError(f"Unexpected cov length for {path}: {arr.size} vs {N_expect*N_expect}")
    return arr

def spd_cholesky_local(C):
    try:
        return cho_factor(C, lower=False, check_finite=False)
    except Exception:
        pass
    for jit in [1e-12,1e-11,1e-10,1e-9]:
        try:
            return cho_factor(C + jit*np.eye(C.shape[0]), lower=False, check_finite=False)
        except Exception:
            pass
    w,V = np.linalg.eigh(C); floor = max(1e-12, 1e-12*abs(np.median(np.diag(C))), 1e-12*abs(np.median(w)))
    w = np.maximum(w, floor); Cc = (V*w)@V.T; Cc = 0.5*(Cc + Cc.T)
    return cho_factor(Cc, lower=False, check_finite=False)

class BAOBlock:
    def __init__(self, y, z, cov=None, sigma=None, obs_type="DV_over_rd"):
        self.y = np.asarray(y,float); self.z = np.asarray(z,float); self.N = len(self.y)
        self.obs_type = obs_type
        if cov is not None:
            C = cov.reshape((self.N,self.N), order="C"); C = 0.5*(C + C.T)
            self.Cf = spd_cholesky_local(C)
            self.Ci = lambda v: cho_solve(self.Cf, v, check_finite=False)
        else:
            if sigma is None: raise ValueError("Provide sigma or cov.")
            w = 1.0/np.square(np.asarray(sigma,float))
            self.Ci = lambda v, w=w: w*v

    def gvec(self, E2_func):
        if self.obs_type == "DV_over_rd":
            return DV_over_rd_g(E2_func, self.z)
        elif self.obs_type == "DM_over_rd":
            return DM_over_rd_g(E2_func, self.z)
        elif self.obs_type == "DH_over_rd":
            return DH_over_rd_g(E2_func, self.z)
        elif self.obs_type.startswith("stack:"):
            kinds = self.obs_type.split(":",1)[1].split(",")
            glist = []
            for k in kinds:
                if k=="DM_over_rd": glist.append(DM_over_rd_g(E2_func, self.z))
                elif k=="DH_over_rd": glist.append(DH_over_rd_g(E2_func, self.z))
                elif k=="DV_over_rd": glist.append(DV_over_rd_g(E2_func, self.z))
                else: raise ValueError(f"Unknown kind {k}")
            return np.concatenate(glist, axis=0)
        else:
            raise ValueError(f"Unknown obs_type {self.obs_type}")

# Build BAO blocks
blocks = []
for e in entries:
    path = Path(e["file"])
    fmt = e.get("format","whitespace").lower()
    if fmt == "csv":
        dfb = pd.read_csv(path)
    else:
        dfb = pd.read_csv(path, sep=r"\s+", engine="python")
    z = dfb[e["z_col"]].to_numpy(float)

    obs_type = e["obs_type"]
    if obs_type.startswith("stack:"):
        cols = e["obs_cols"]
        y = np.concatenate([dfb[c].to_numpy(float) for c in cols], axis=0)
        N = len(y)
    else:
        y = dfb[e["obs_col"]].to_numpy(float)
        N = len(y)

    cov_path = e.get("cov", None)
    sigma_col = e.get("sigma_col", None)
    if cov_path:
        flat = load_matrix(Path(cov_path), N_expect=N)
        block = BAOBlock(y=y, z=z, cov=flat, obs_type=obs_type)
    else:
        if sigma_col is None: raise SystemExit(f"[BAO] Entry requires sigma_col or cov: {e}")
        sig = dfb[sigma_col].to_numpy(float)
        block = BAOBlock(y=y, z=z, sigma=sig, obs_type=obs_type)
    blocks.append(block)

print(f"[BAO] Loaded {len(blocks)} block(s).")

# ---- Shared helpers for model runs ----
def mu_SN(E2_func): return mu_from_E2_grid(E2_func, z_sn, n_grid=20000)

def chi2_BAO_global_alpha(E2_func):
    # Global alpha marginalized across all blocks:
    # chi2 = sum(y^T C^-1 y) - ( (sum g^T C^-1 y)^2 ) / (sum g^T C^-1 g )
    C0 = 0.0; A = 0.0; B = 0.0
    for b in blocks:
        y = b.y; g = b.gvec(E2_func)
        C0 += float(y @ b.Ci(y))
        A  += float(g @ b.Ci(g))
        B  += float(g @ b.Ci(y))
    chi2 = float(C0 - (B*B)/max(A, 1e-300))
    alpha_hat = float(B / max(A, 1e-300))
    return chi2, alpha_hat, {"C0": C0, "A": A, "B": B}

# ---- Runs: UAM (k=0), LCDM (k=1) ----
COARSE_OM = np.linspace(0.10, 0.50, 401)
REFINE_SPAN, REFINE_STEPS = 0.02, 81

def run_uam():
    mu = mu_SN(E2_uam); c2_sn = chi2_sn_marg(mu)
    c2_bao, a_hat, parts = chi2_BAO_global_alpha(E2_uam)
    return {"chi2_sn": c2_sn, "chi2_bao": c2_bao, "chi2_tot": c2_sn+c2_bao,
            "alpha_hat": a_hat, "bao_parts": parts}

def run_lcdm():
    best = (None, np.inf, None, None, None)
    for om in COARSE_OM:
        E2 = E2_lcdm_factory(float(om))
        mu = mu_SN(E2); c2_sn = chi2_sn_marg(mu)
        c2_bao, a_hat, parts = chi2_BAO_global_alpha(E2)
        c2 = c2_sn + c2_bao
        if c2 < best[1]: best = (float(om), c2, c2_sn, c2_bao, a_hat)
    om1 = best[0]; lo, hi = max(0.10,om1-REFINE_SPAN), min(0.50,om1+REFINE_SPAN)
    for om in np.linspace(lo, hi, REFINE_STEPS):
        E2 = E2_lcdm_factory(float(om))
        mu = mu_SN(E2); c2_sn = chi2_sn_marg(mu)
        c2_bao, a_hat, parts = chi2_BAO_global_alpha(E2)
        c2 = c2_sn + c2_bao
        if c2 < best[1]: best = (float(om), c2, c2_sn, c2_bao, a_hat)
    om_b, c2_tot, c2_sn, c2_bao, alpha_hat = best
    return {"Omega_m": om_b, "chi2_sn": c2_sn, "chi2_bao": c2_bao, "chi2_tot": c2_tot,
            "alpha_hat": alpha_hat}

U = run_uam()
L = run_lcdm()

# ---- Model comparison ----
N_params = {"UAM":0, "LCDM":1}
def aic(c2,k): return c2 + 2*k
def bic(c2,k,n): return c2 + k*math.log(max(n,1))

# Effective N for selection: SN contributes N_sn, BAO contributes total observed elements across blocks
N_bao = int(sum(b.N for b in blocks))
N_tot = int(N_sn + N_bao)

tab = pd.DataFrame([
  ["UAM",  "-",     N_params["UAM"], U["chi2_sn"], U["chi2_bao"], U["chi2_tot"], aic(U["chi2_tot"],0), bic(U["chi2_tot"],0,N_tot), U["alpha_hat"]],
  ["LCDM", f"{L['Omega_m']:.3f}", 1, L["chi2_sn"], L["chi2_bao"], L["chi2_tot"], aic(L["chi2_tot"],1), bic(L["chi2_tot"],1,N_tot), L["alpha_hat"]],
], columns=["Model","Omega_m","k","chi2_SN","chi2_BAO","chi2_tot","AIC","BIC","alpha_hat"])

print(tab.to_string(index=False))

# ---- Log JSON ----
log = {
  "timestamp_utc": time.strftime("%Y-%m-%d %H:%M:%S"),
  "env": {"python": sys.version, "numpy": np.__version__, "pandas": pd.__version__, "platform": platform.platform()},
  "sn": {"N": N_sn, "cov_spd": spd_sn, "Delta_hat_UAM": float(delta_hat(mu_SN(E2_uam)))},
  "bao": {"N": N_bao, "blocks": len(blocks), "global_alpha": True},
  "results": {"UAM": U, "LCDM": L, "table": tab.to_dict(orient="records")}
}
(LOG_DIR/"SNplusBAO_09012025.json").write_text(json.dumps(log, indent=2), encoding="utf-8")

# ---- Report addendum ----
rep = ROOT/"reports/SN_UAM_vs_LCDM_09012025.md"
md = (
    "\n\n## SN + BAO (deterministic; Delta and global alpha marginalized)\n\n"
    f"Data counts: SN={N_sn}, BAO={N_bao} (blocks={len(blocks)}); total={N_tot}.\n\n"
    f"LambdaCDM best-fit Omega_m={L['Omega_m']:.3f}; global BAO alpha hats — UAM={U['alpha_hat']:.5f}, LCDM={L['alpha_hat']:.5f}.\n\n"
    "| Model | Omega_m | k | chi2_SN | chi2_BAO | chi2_tot | AIC | BIC |\n"
    "|---|---:|---:|---:|---:|---:|---:|---:|\n"
    f"| UAM  | -   | 0 | {U['chi2_sn']:.2f} | {U['chi2_bao']:.2f} | {U['chi2_tot']:.2f} | {aic(U['chi2_tot'],0):.2f} | {bic(U['chi2_tot'],0,N_tot):.2f} |\n"
    f"| LCDM | {L['Omega_m']:.3f} | 1 | {L['chi2_sn']:.2f} | {L['chi2_bao']:.2f} | {L['chi2_tot']:.2f} | {aic(L['chi2_tot'],1):.2f} | {bic(L['chi2_tot'],1,N_tot):.2f} |\n\n"
    f"_Log_: {LOG_DIR/'SNplusBAO_09012025.json'}\n"
)
try:
    txt = rep.read_text(encoding="utf-8")
except FileNotFoundError:
    txt = "# SN Baseline — UAM vs LCDM\n"
rep.write_text(txt.rstrip()+"\n"+md, encoding="utf-8")
print("[log]", LOG_DIR/'SNplusBAO_09012025.json')
print("[report updated]", rep)
